In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch5
(    
  mch5_id STRING,
  mch5_name STRING,
  mch5_desc STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 """)

In [0]:
spark.sql(f"""CREATE OR REPLACE TEMP VIEW temp_d_mch5 AS 
SELECT 
a._bic_zc_mchlv5 mch5_id,  
a.txtsh mch5_name,
a.txtmd mch5_desc
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY _bic_zc_mchlv5 ORDER BY proc_date DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_zc_mchlv5
        WHERE end_date = '2400-01-01'
        AND LENGTH(_bic_zc_mchlv5) = 9
        AND REGEXP(_bic_zc_mchlv5, '^[0-9]+$')
) a WHERE rn = 1""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch5 AS main_tbl
USING temp_d_mch5 AS temp_tbl
ON main_tbl.mch5_id = temp_tbl.mch5_id

WHEN MATCHED THEN 
    UPDATE SET 
        main_tbl.mch5_name = temp_tbl.mch5_name,
        main_tbl.mch5_desc = temp_tbl.mch5_desc

WHEN NOT MATCHED THEN 
    INSERT (mch5_id, mch5_name, mch5_desc)
    VALUES (temp_tbl.mch5_id, temp_tbl.mch5_name, temp_tbl.mch5_desc)
""")

In [0]:
spark.sql("DROP VIEW IF EXISTS temp_d_mch5")
